In [12]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load dataset
vocab_size = 10000
maxlen = 200

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=32, input_length=maxlen),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train,
          epochs=10,
          batch_size=512,
          validation_split=0.2,
          verbose=2)

# Evaluate
loss, accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"\nTest Accuracy: {accuracy:.4f}")

# Load word index for decoding
word_index = imdb.get_word_index()
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3
reverse_word_index = {value: key for (key, value) in word_index.items()}

# Function to encode a custom review
def encode_review(text):
    tokens = text.lower().split()
    encoded = [1]  # Start token
    for word in tokens:
        encoded.append(word_index.get(word, 2))  # 2 is UNK (unknown)
    return pad_sequences([encoded], maxlen=maxlen)

# Function to predict review sentiment
def predict_sentiment(review_text):
    encoded = encode_review(review_text)
    pred = model.predict(encoded)[0][0]
    label = "positive" if pred >= 0.5 else "negative"
    print(f"Review: {review_text}")
    print(f"Predicted sentiment: {label} ({pred:.2f})\n")

# Test with custom reviews
predict_sentiment("The film was amazing.")
predict_sentiment("This was the worst film I have ever seen")


Epoch 1/10
40/40 - 2s - 45ms/step - accuracy: 0.6054 - loss: 0.6848 - val_accuracy: 0.7006 - val_loss: 0.6685
Epoch 2/10
40/40 - 1s - 16ms/step - accuracy: 0.7321 - loss: 0.6454 - val_accuracy: 0.7392 - val_loss: 0.6106
Epoch 3/10
40/40 - 1s - 16ms/step - accuracy: 0.7859 - loss: 0.5699 - val_accuracy: 0.8048 - val_loss: 0.5253
Epoch 4/10
40/40 - 1s - 17ms/step - accuracy: 0.8292 - loss: 0.4804 - val_accuracy: 0.8344 - val_loss: 0.4470
Epoch 5/10
40/40 - 1s - 17ms/step - accuracy: 0.8493 - loss: 0.4072 - val_accuracy: 0.8498 - val_loss: 0.3918
Epoch 6/10
40/40 - 1s - 16ms/step - accuracy: 0.8687 - loss: 0.3522 - val_accuracy: 0.8424 - val_loss: 0.3674
Epoch 7/10
40/40 - 1s - 17ms/step - accuracy: 0.8778 - loss: 0.3191 - val_accuracy: 0.8588 - val_loss: 0.3373
Epoch 8/10
40/40 - 1s - 17ms/step - accuracy: 0.8870 - loss: 0.2935 - val_accuracy: 0.8704 - val_loss: 0.3226
Epoch 9/10
40/40 - 1s - 17ms/step - accuracy: 0.8976 - loss: 0.2707 - val_accuracy: 0.8748 - val_loss: 0.3125
Epoch 10/1